In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import string

In [2]:
import glob
from tqdm import tqdm 
#import pdfminer
from pdfminer.high_level import extract_text
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk 
nltk.download('stopwords')
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sunilmcesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.probability import FreqDist

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sunilmcesh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sunilmcesh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
PAPER_OF_INTEREST_FNAME  =  glob.glob('/Volumes/Seagate Backup Plus Drive/JOSS Project/joss-papers-master/*/*/*.pdf')

In [5]:
# https://zenodo.org/record/3631674#.YeglfP7P2Uk
df = pd.read_csv('/Volumes/Seagate Backup Plus Drive/JOSS Project/wiki_tfidf_terms.csv')
df_reviewers = pd.read_csv('../Data/JOSS Table Test.csv')

In [6]:
def Get_Lemma_Words(POI_PDF):
  text = str(POI_PDF)
  text2 = text.split()
  words_no_punc = []

  for w in text2:
    if w.isalpha():
      words_no_punc.append(w.lower())
  from nltk.corpus import stopwords
  stopwords = stopwords.words('english')  
  clean_words = []
  for w in words_no_punc:
    if w not in stopwords:
      clean_words.append(w)
  clean_words_arr = ''
  for i in range(len(clean_words)):
    clean_words_arr = clean_words_arr + ' ' + str(clean_words[i])

  string_for_lemmatizing = clean_words_arr
  lemmatizer = WordNetLemmatizer()
  words_2 = word_tokenize(string_for_lemmatizing)
  lemmatized_words = [lemmatizer.lemmatize(word) for word in words_2]

  lemmatized_words_arr = ''
  for i in range(len(lemmatized_words)):
    lemmatized_words_arr = lemmatized_words_arr + ' ' + str(lemmatized_words[i])
  words = word_tokenize(lemmatized_words_arr)

  return words

In [7]:
def Get_reviewer_sample_tf_idf(Paper_interest, df=df, df_reviewers=df_reviewers, num_suggestions=5, num_top20=20):
    #POI_PDF = [extract_text(Paper_interest)]
    #text = str(POI_PDF)
    #words =  Get_Lemma_Words(POI_PDF)
    ##print(len(words))
    #fdist = FreqDist(words)
    #X = np.array(fdist.most_common())
    #top20_tf = X[:num_top20,0]
    #match_arr = Compare_topics(top20_tf, df_reviewers)
    #top5_reviewers = np.argsort(match_arr)[-num_suggestions:]

    POI_PDF = [extract_text(Paper_interest)]
    text = str(POI_PDF)
    words = Get_Lemma_Words(POI_PDF)
    #print(len(words))
    fdist = FreqDist(words)
    X = np.array(fdist.most_common())
    tf_idf_arr_names, tf_idf_arr_floats = determine_wiki_td_idf(X, df=df)
    #print('determined wiki')
    num_arr = np.array(tf_idf_arr_floats)
    tf_idf_arr_names_arr = np.array(tf_idf_arr_names)
    top20_tf_idf = tf_idf_arr_names_arr[np.argsort(num_arr)[:20]]
    match_arr = Compare_topics(top20_tf_idf, df_reviewers)
    top5_reviewers = np.argsort(match_arr)[-num_suggestions:]
    
    
    all_usernames = []
    all_domains = []
    all_num_matched_words = []
    all_matched_words = []
    for i in range(num_suggestions):
      K = -1*(i+1)
      index = top5_reviewers[K]
      #print(i)
      t =df_reviewers.iloc[index+1]['Domains/topic areas you are comfortable reviewing'].lower()
      
      all_usernames.append(df_reviewers.username.iloc[index+1])
      all_domains.append(t)
      all_num_matched_words.append(match_arr[index])

      uniarr = Split_columns(t)
      matched_words = []
      #print(uniarr)
      for j in range(len(uniarr)):
        for k in range(len(top20_tf_idf)):
          if uniarr[j] == top20_tf_idf[k]:
            matched_words.append(uniarr[j])
      all_matched_words.append(matched_words)

    #df_reviewers.username.iloc[+1]

    return all_usernames, all_domains, all_num_matched_words, all_matched_words

In [8]:
def Compare_topics(top20, df_reviewers):
  length = df_reviewers.shape[0] - 1
  match_arr = np.zeros(length)
  for i in range(length):
    if pd.isna(df_reviewers['Domains/topic areas you are comfortable reviewing'].str.lower().values[1+i]) == False:
      t = df_reviewers['Domains/topic areas you are comfortable reviewing'].str.lower().values[1+i]
      #print(i)
      uniarr = Split_columns(t)
      for j in range(len(uniarr)):
        for k in range(len(top20)):
          if uniarr[j] == top20[k]:
            match_arr[i] = match_arr[i] + 1
  return match_arr

In [9]:
def Split_columns(t):
  txt = " ".join("".join([" " if ch in string.punctuation else ch for ch in t]).split())
  sol1 = np.char.split(txt, ' ')
  txt_arr  = array_of_lists_to_array(sol1)
  uniarr = np.unique(txt_arr)
  return uniarr

In [10]:
 def array_of_lists_to_array(arr):
    return np.apply_along_axis(lambda a: np.array(a[0]), -1, arr[..., None])

In [11]:
def determine_wiki_td_idf(x, df=df):
    tf_idf_arr_names = []
    tf_idf_arr_floats = []
    for i in range(len(x)):
        if df[df['token'] ==x[i][0]].frequency.empty == False:
            wiki_tf = df[df['token'] ==x[i][0]].frequency.values[0]
            doc_tf = int(x[i][1])
            tf_idf = np.log(wiki_tf/doc_tf)
            tf_idf_arr_names.append(x[i][0])
            tf_idf_arr_floats.append(tf_idf)
    return tf_idf_arr_names, tf_idf_arr_floats

In [12]:
def summatation_bot(all_usernames, all_domains, all_num_matched_words, all_matched_words):
  length = len(all_usernames)
  message = 'Hello. \nI have found ' + str(length) + ' possible reviewers for this paper.' +'\n\n'
  for i in range(length):
    ps = 'I believe ' + all_usernames[i] + ' will make a good reviewer for this paper because they have matched ' + str(int(all_num_matched_words[i])) +  ' words from their comfortable domain topics with the top 20 most frequent words in the paper. These matched words are ' + str(all_matched_words[i]) +'.\nFrom their topics domain: ' + str(all_domains[i].replace('\n', ', ')) +'.\n'
    message = message + ps + '\n'
  print(message)

In [13]:
Q = 340
print(PAPER_OF_INTEREST_FNAME[Q])

/Volumes/Seagate Backup Plus Drive/JOSS Project/joss-papers-master/joss-papers-master/joss.00775/10.21105.joss.00775.pdf


In [14]:
all_usernames, all_domains, all_num_matched_words, all_matched_words = Get_reviewer_sample_tf_idf(PAPER_OF_INTEREST_FNAME[Q])

In [15]:
summatation_bot(all_usernames, all_domains, all_num_matched_words, all_matched_words)

Hello. 
I have found 5 possible reviewers for this paper.

I believe csadorf will make a good reviewer for this paper because they have matched 2 words from their comfortable domain topics with the top 20 most frequent words in the paper. These matched words are ['github', 'software'].
From their topics domain: data management,  workflow management,  molecular dynamics,  metropolis monte carlo,  crystallization,  machine learning,  neural networks,  parallelism,  cuda,  mpi,  high-performance clusters,  supercomputers,  github actions,  circle ci,  pandas,  project jupyter,  materials science,  chemical engineering,  scientific software.

I believe jarrah42 will make a good reviewer for this paper because they have matched 2 words from their comfortable domain topics with the top 20 most frequent words in the paper. These matched words are ['software', 'user'].
From their topics domain: high performance/scientific computing,  operating systems,  distributed systems,  parallel programmi